## Import Libraries

In [1]:
!pip install tqdm

    100% |████████████████████████████████| 71kB 10.5MB/s ta 0:00:01
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from tqdm.auto import tqdm 
from tqdm import tqdm_notebook

## Paths

In [3]:
msmarco_dir = "../data/msmarco_files/"
anserini_dir = "../data/anserini_output/"
output_dir = "../data/output/"

## Load Data

In [4]:
relevance_df = pd.read_csv(msmarco_dir + 'qrels.dev.small.tsv',delimiter='\t',encoding='utf-8',header=None)
relevance_df.columns = ['query_id','label1','passage_id','label2']
relevance_df = relevance_df.drop(columns=['label1','label2'])

In [5]:
bert_filenames = [f for f in listdir(output_dir) if (isfile(join(output_dir, f))) and (f.startswith("bert_run_msmarco_leaderboard"))]

In [6]:
bert_filenames

['bert_run_msmarco_leaderboard_df_subset_9.tsv',
 'bert_run_msmarco_leaderboard_df_subset_8.tsv',
 'bert_run_msmarco_leaderboard_df_subset_24.tsv',
 'bert_run_msmarco_leaderboard_df_subset_18.tsv',
 'bert_run_msmarco_leaderboard_df_subset_10.tsv',
 'bert_run_msmarco_leaderboard_df_subset_11.tsv',
 'bert_run_msmarco_leaderboard_df_subset_13.tsv',
 'bert_run_msmarco_leaderboard_df_subset_17.tsv',
 'bert_run_msmarco_leaderboard_df_subset_21.tsv',
 'bert_run_msmarco_leaderboard_df_subset_28.tsv',
 'bert_run_msmarco_leaderboard_df_subset_3.tsv',
 'bert_run_msmarco_leaderboard_df_subset_6.tsv',
 'bert_run_msmarco_leaderboard_df_subset_27.tsv',
 'bert_run_msmarco_leaderboard_df_subset_14.tsv',
 'bert_run_msmarco_leaderboard_df_subset_20.tsv',
 'bert_run_msmarco_leaderboard_df_subset_22.tsv',
 'bert_run_msmarco_leaderboard_df_subset_23.tsv',
 'bert_run_msmarco_leaderboard_df_subset_4.tsv',
 'bert_run_msmarco_leaderboard_df_subset_5.tsv',
 'bert_run_msmarco_leaderboard_df_subset_12.tsv',
 'bert

Note: it seems as you are missing a file (file 2). But file 1 contains both 1 & 2, after this file I decided to make even smaller subsets. Which is why I continued with 3. Files 3 an onward contain 250 queries. While file 1 contains 500 queries.

In [7]:
bert_dfs = []
for bert_filename in tqdm_notebook(bert_filenames):
    bert_df = pd.read_csv(output_dir + bert_filename,delimiter='\t',encoding='utf-8', header=None)
    bert_df.columns = ['query_id', 'passage_id', 'bm25_rank', 'query', 'passage', 'bert_score', 'bert_rank']
    bert_dfs.append(bert_df)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


## Make 1 big DataFrame

In [8]:
bert_rankings = pd.concat(bert_dfs, ignore_index=True)

In [9]:
bert_rankings.to_csv(output_dir + 'bert_run_msmarco_leaderboard_df_all_subsets.tsv',sep="\t", header=False,index=False)

In [5]:
bert_rankings = pd.read_csv(output_dir + 'bert_run_msmarco_leaderboard_df_all_subsets.tsv',delimiter='\t',encoding='utf-8', header=None)
bert_rankings.columns = ['query_id', 'passage_id', 'bm25_rank', 'query', 'passage', 'bert_score', 'bert_rank']

## Compute MRR@10

In [6]:
def compute_mrr(gt, pred):
    score = 0.0
    best_rank = 11
    for index, row in pred.iterrows():
        current_rank = row['bert_rank']
        if row['passage_id'] in gt:
            if current_rank < best_rank:
                score = 1.0 / (row['bert_rank'])
                best_rank = current_rank
    return score

In [ ]:
mrr = 0.0
query_ids = list(np.unique(bert_rankings['query_id'].tolist()))
relevance_df = relevance_df[relevance_df['query_id'].isin(query_ids)]
for query_id in tqdm_notebook(query_ids):
    query_preds_df = bert_rankings[(bert_rankings['query_id'] == query_id) & (bert_rankings['bert_rank'] < 11)]
    gt = relevance_df[relevance_df['query_id'] == query_id]['passage_id'].values.tolist()
    mrr += compute_mrr(gt, query_preds_df)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [16]:
round((mrr/len(query_ids))*100,1)

34.7